# Análise de Proteínas

## 1. Processamento do arquivo original

### Importação das libs

In [41]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import re
import itertools
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, MeanShift, AffinityPropagation
from sklearn.metrics import silhouette_score, calinski_harabasz_score, f1_score, adjusted_mutual_info_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder
import warnings

### Processamento do Arquivo

In [2]:
arquivo = "proteinas.fa"

headers = []
seqs = []
classes = []

for registro in SeqIO.parse(arquivo, "fasta"):
    headers.append(registro.id)
    seqs.append(str(registro.seq))    
    classes.append(registro.description)

### DataFrame

In [3]:
data_proteinas = pd.DataFrame({
    'header': headers,
    'sequencia': seqs,
    'classe_scop': classes
})

print(f'Total de proteínas: {len(data_proteinas)}')
print(f'Comprimento medio: {data_proteinas['sequencia'].str.len().mean():.2f} aminoácidos.')
print(f'Menor comprimento: {data_proteinas['sequencia'].str.len().min()} aminoácidos.')
print(f'Maior comprimento: {data_proteinas['sequencia'].str.len().max()} aminoácidos.')

print()
print('Primeiras 5 entradas:')
data_proteinas.head()

Total de proteínas: 15177
Comprimento medio: 184.82 aminoácidos.
Menor comprimento: 20 aminoácidos.
Maior comprimento: 1664 aminoácidos.

Primeiras 5 entradas:


,header,sequencia,classe_scop
0,d1dlwa_,slfeqlggqaavqavtaqfyaniqadatvatffngidmpnqtnkta...,d1dlwa_ a.1.1.1 (A:) Protozoan/bacterial hemog...
1,d2gkma_,gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,d2gkma_ a.1.1.1 (A:) Protozoan/bacterial hemog...
2,d1ngka_,ksfydavggaktfdaivsrfyaqvaedevlrrvypeddlagaeerl...,d1ngka_ a.1.1.1 (A:) Protozoan/bacterial hemog...
3,d2bkma_,eqwqtlyeaiggeetvaklveafyrrvaahpdlrpifpddltetah...,d2bkma_ a.1.1.1 (A:) automated matches {Geobac...
4,d4i0va_,aslyeklggaaavdlavekfygkvladervnrffvntdmakqkqhq...,d4i0va_ a.1.1.1 (A:) automated matches {Synech...


## 2. Extração das classes SCOP

In [4]:
scop_padrao = r'([a-z]\.[0-9]+\.[0-9]+\.[0-9]+)'

### Método para extração e aplicação

In [5]:
def extrair_classe_scop(descricao):
    match = re.search(scop_padrao, descricao)
    if match:
        return match.group(1)
    else:
        return None
    
data_proteinas['classe_scop'] = data_proteinas['classe_scop'].apply(extrair_classe_scop)

### Visualização dos resultados:
Resumo da Extração:

In [6]:
resumo_extracao = pd.DataFrame({
    'Metrica':[
        'Total de Sequências',
        'Com classe SCOP identificada',
        'Sem classe SCOP',
        'Taxa de Sucesso',
        'Classes únicas identificadas'
    ],
    'Valor':[
        len(data_proteinas),
        data_proteinas['classe_scop'].notna().sum(),
        data_proteinas['classe_scop'].isna().sum(),
        f'{(data_proteinas['classe_scop'].notna().sum()/len(data_proteinas)*100):.2f}%',
        data_proteinas['classe_scop'].nunique()
    ]        
})

resumo_extracao.head()

,Metrica,Valor
0,Total de Sequências,15177
1,Com classe SCOP identificada,15177
2,Sem classe SCOP,0
3,Taxa de Sucesso,100.00%
4,Classes únicas identificadas,4703


Distribuição das Classes:

In [7]:
distribuicao_classes = data_proteinas['classe_scop'].value_counts().head()
df_distrubuicao = pd.DataFrame({
    'Classe SCOP': distribuicao_classes.index,
    'Quantidade': distribuicao_classes.values,
    'Percentual': (distribuicao_classes.values / len(data_proteinas)*100).round(2)
})

df_distrubuicao

,Classe SCOP,Quantidade,Percentual
0,c.2.1.0,196,1.29
1,c.94.1.0,132,0.87
2,b.1.1.0,127,0.84
3,c.47.1.0,121,0.80
4,c.37.1.0,109,0.72


Amostra dos dados com classes extraídas:

In [8]:
amostra = data_proteinas[['header', 'classe_scop', 'sequencia']].head(11).copy()
amostra['comprimento'] = amostra['sequencia'].str.len()

amostra

,header,classe_scop,sequencia,comprimento
0,d1dlwa_,a.1.1.1,slfeqlggqaavqavtaqfyaniqadatvatffngidmpnqtnkta...,116
1,d2gkma_,a.1.1.1,gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,127
2,d1ngka_,a.1.1.1,ksfydavggaktfdaivsrfyaqvaedevlrrvypeddlagaeerl...,126
3,d2bkma_,a.1.1.1,eqwqtlyeaiggeetvaklveafyrrvaahpdlrpifpddltetah...,128
4,d4i0va_,a.1.1.1,aslyeklggaaavdlavekfygkvladervnrffvntdmakqkqhq...,123
5,d1asha_,a.1.1.2,anktrelcmkslehakvdtsnearqdgidlykhmfenypplrkyfk...,147
6,d2dc3a_,a.1.1.2,eelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsq...,172
7,d4hswa_,a.1.1.2,gfkqdiatirgdlrtyaqdiflaflnkypderryfknyvgksdqel...,137
8,d1ecaa_,a.1.1.2,lsadqistvqasfdkvkgdpvgilyavfkadpsimakftqfagkdl...,136
9,d1x9fd_,a.1.1.2,eclvteslkvklqwasafghahervafglelwrdiiddhpeikapf...,140


Estatísticas Extras:

In [9]:
distribuicao_classes = data_proteinas['classe_scop'].value_counts()

estatisicas_classes = pd.DataFrame({
    'Estatística': [
        'Média de sequências por classe',
        'Classe com mais sequências',
        'Classe com menos sequências',
        'Classes com apenas uma sequência'
    ],
    'Valor': [
        f'{data_proteinas['classe_scop'].value_counts().mean():.1f}',
        f"{distribuicao_classes.index[0]} ({distribuicao_classes.iloc[0]} sequências)",
        f'{data_proteinas['classe_scop'].value_counts().min()}',
        f'{(data_proteinas['classe_scop'].value_counts() == 1).sum()}'
    ]
})

estatisicas_classes

,Estatística,Valor
0,Média de sequências por classe,3.2
1,Classe com mais sequências,c.2.1.0 (196 sequências)
2,Classe com menos sequências,1
3,Classes com apenas uma sequência,2454


## 3. Implementação do K-mer 2x2

In [10]:
aminoacidos = 'ACDEFGHIKLMNPQRSTVWY'
len(aminoacidos)

20

### Gerando pares possíveis de K-mers

In [11]:
primeiros = [f'{a}{b}' for a in aminoacidos for b in aminoacidos]
segundos = [f'{c}{d}' for c in aminoacidos for d in aminoacidos]

lista_kmers_2x2 = [f'{p1}_{p2}' for p1 in primeiros for p2 in segundos]

print(f'Total de k-mers 2x2 possíveis: {len(lista_kmers_2x2)}')
print(f'Ex: {lista_kmers_2x2[:5]}')

Total de k-mers 2x2 possíveis: 160000
Ex: ['AA_AA', 'AA_AC', 'AA_AD', 'AA_AE', 'AA_AF']


### Método para extrair K-mers das sequências

In [12]:
def extracao_kmers(sequencia):
    kmers_achados = set()
    comprimento = len(sequencia)
    
    for i in range(comprimento - 4): # pq são 5
        kmer = f'{sequencia[i]}{sequencia[i+1]}_{sequencia[i+3]}{sequencia[i+4]}'
        kmers_achados.add(kmer.upper())
            
    return kmers_achados

### Criando a Matriz Binária
Dicionário de índice para os k-mers:

In [13]:
kmer_to_indice = {kmer: idx for idx, kmer in enumerate(lista_kmers_2x2)}

Método para criar vetor binários de K-mers para uma sequência:

In [14]:
def vet_kmers(sequencia, k_to_i):
    vetor = np.zeros(len(k_to_i), dtype=int)
    kmers_achados = extracao_kmers(sequencia)
    
    for kmer in kmers_achados:
        if kmer in k_to_i:
            vetor[k_to_i[kmer]] = 1
            
    return vetor

Gerando as matrizes

In [15]:
lote = data_proteinas['sequencia'].head(3000).tolist()

matriz_k = []
for seq in tqdm(lote, desc='Gerando matrizes...'):
    vetor = vet_kmers(seq, kmer_to_indice)
    matriz_k.append(vetor)
    
matriz_kmers = np.array(matriz_k)
print(f'Matriz n°1 criada: {matriz_kmers.shape}')

Gerando matrizes...: 100%|██████████| 3000/3000 [00:01<00:00, 1988.80it/s]


Matriz n°1 criada: (3000, 160000)


In [16]:
print(f"Densidade da matriz: {(matriz_kmers.sum() / (matriz_kmers.shape[0] * matriz_kmers.shape[1]) * 100):.4f}%")

Densidade da matriz: 0.0900%


## 4. PCA

In [17]:
pca = PCA(n_components=300)
matriz_pca = pca.fit_transform(matriz_kmers)

print(f'Matriz pós PCA: {matriz_pca.shape}')
print(f'Variância explicada total: {pca.explained_variance_ratio_.sum():.4f}')

Matriz pós PCA: (3000, 300)
Variância explicada total: 0.2884


## 5. Clustering

In [26]:
x = matriz_pca
classes_verdadeiras = data_proteinas['classe_scop'].head(3000).values

resultados = {}

1. KMeans:

In [42]:
kmeans = KMeans(n_clusters=15, random_state=42)
labels_kmeans = kmeans.fit_predict(x)

le = LabelEncoder()
classes_numericas = le.fit_transform(classes_verdadeiras)

n_clusters_kmeans = len(set(labels_kmeans))
sil_kmeans = silhouette_score(x, labels_kmeans)
ch_kmeans = calinski_harabasz_score(x, labels_kmeans)
ari_kmeans = adjusted_rand_score(classes_numericas, labels_kmeans)
ami_kmeans = adjusted_mutual_info_score(classes_numericas, labels_kmeans)

print(f"Clusters: {n_clusters_kmeans}")
print(f"Silhouette: {sil_kmeans:.4f}")
print(f"Calinski-Harabasz: {ch_kmeans:.2f}")
print(f"ARI: {ari_kmeans:.4f}")
print(f"AMI: {ami_kmeans:.4f}")

resultado_kmeans = {
    'Algoritmo': 'KMeans',
    'N_Clusters': n_clusters_kmeans,
    'Silhouette': sil_kmeans,
    'Calinski_Harabasz': ch_kmeans,
    'ARI': ari_kmeans,
    'AMI': ami_kmeans
}

Clusters: 15
Silhouette: 0.7578
Calinski-Harabasz: 16.26
ARI: 0.0001
AMI: 0.0010


2. DBSCAN:

In [48]:
dbscan = DBSCAN(eps=1.0, min_samples=5)
labels_dbscan = dbscan.fit_predict(x)

le = LabelEncoder()
classes_numericas = le.fit_transform(classes_verdadeiras)

n_clusters_dbscan = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_ruido = list(labels_dbscan).count(-1)

if n_clusters_dbscan > 1:
    sil_dbscan = silhouette_score(x, labels_dbscan)
    ch_dbscan = calinski_harabasz_score(x, labels_dbscan)
else:
    sil_dbscan = -1
    ch_dbscan = -1

ari_dbscan = adjusted_rand_score(classes_numericas, labels_dbscan)
ami_dbscan = adjusted_mutual_info_score(classes_numericas, labels_dbscan)

print(f"Clusters: {n_clusters_dbscan} | Ruído: {n_ruido}")
print(f"Silhouette: {sil_dbscan:.4f}")
print(f"Calinski-Harabasz: {ch_dbscan:.2f}")
print(f"ARI: {ari_dbscan:.4f}")
print(f"AMI: {ami_dbscan:.4f}")

resultado_dbscan = {
    'Algoritmo': 'DBSCAN',
    'N_Clusters': n_clusters_dbscan,
    'Silhouette': sil_dbscan,
    'Calinski_Harabasz': ch_dbscan,
    'ARI': ari_dbscan,
    'AMI': ami_dbscan
}

Clusters: 1 | Ruído: 1628
Silhouette: -1.0000
Calinski-Harabasz: -1.00
ARI: 0.0074
AMI: 0.0863


3. Agglomerative Clustering:

In [49]:
agglo = AgglomerativeClustering(n_clusters=15)
labels_agglo = agglo.fit_predict(x)

le = LabelEncoder()
classes_numericas = le.fit_transform(classes_verdadeiras)

n_clusters_agglo = len(set(labels_agglo))
sil_agglo = silhouette_score(x, labels_agglo)
ch_agglo = calinski_harabasz_score(x, labels_agglo)
ari_agglo = adjusted_rand_score(classes_numericas, labels_agglo)
ami_agglo = adjusted_mutual_info_score(classes_numericas, labels_agglo)

print(f"Clusters: {n_clusters_agglo}")
print(f"Silhouette: {sil_agglo:.4f}")
print(f"Calinski-Harabasz: {ch_agglo:.2f}")
print(f"ARI: {ari_agglo:.4f}")
print(f"AMI: {ami_agglo:.4f}")

resultado_agglo = {
    'Algoritmo': 'Agglomerative',
    'N_Clusters': n_clusters_agglo,
    'Silhouette': sil_agglo,
    'Calinski_Harabasz': ch_agglo,
    'ARI': ari_agglo,
    'AMI': ami_agglo
}

Clusters: 15
Silhouette: 0.7716
Calinski-Harabasz: 22.74
ARI: 0.0014
AMI: 0.0317


4. MeanShift:

In [50]:
meanshift = MeanShift()
labels_meanshift = meanshift.fit_predict(x)

le = LabelEncoder()
classes_numericas = le.fit_transform(classes_verdadeiras)

n_clusters_meanshift = len(set(labels_meanshift))
if n_clusters_meanshift > 1:
    sil_meanshift = silhouette_score(x, labels_meanshift)
    ch_meanshift = calinski_harabasz_score(x, labels_meanshift)
else:
    sil_meanshift = -1
    ch_meanshift = -1

ari_meanshift = adjusted_rand_score(classes_numericas, labels_meanshift)
ami_meanshift = adjusted_mutual_info_score(classes_numericas, labels_meanshift)

print(f"Clusters: {n_clusters_meanshift}")
print(f"Silhouette: {sil_meanshift:.4f}")
print(f"Calinski-Harabasz: {ch_meanshift:.2f}")
print(f"ARI: {ari_meanshift:.4f}")
print(f"AMI: {ami_meanshift:.4f}")

resultado_meanshift = {
    'Algoritmo': 'MeanShift',
    'N_Clusters': n_clusters_meanshift,
    'Silhouette': sil_meanshift,
    'Calinski_Harabasz': ch_meanshift,
    'ARI': ari_meanshift,
    'AMI': ami_meanshift
}

Clusters: 485
Silhouette: 0.4734
Calinski-Harabasz: 161.87
ARI: 0.0031
AMI: 0.0398


5. AffinityPropagation:

In [51]:
affinity = AffinityPropagation(random_state=42, damping=0.9)
labels_affinity = affinity.fit_predict(x)

le = LabelEncoder()
classes_numericas = le.fit_transform(classes_verdadeiras)

n_clusters_affinity = len(set(labels_affinity))
if n_clusters_affinity > 1:
    sil_affinity = silhouette_score(x, labels_affinity)
    ch_affinity = calinski_harabasz_score(x, labels_affinity)
else:
    sil_affinity = -1
    ch_affinity = -1

ari_affinity = adjusted_rand_score(classes_numericas, labels_affinity)
ami_affinity = adjusted_mutual_info_score(classes_numericas, labels_affinity)

print(f"Clusters: {n_clusters_affinity}")
print(f"Silhouette: {sil_affinity:.4f}")
print(f"Calinski-Harabasz: {ch_affinity:.2f}")
print(f"ARI: {ari_affinity:.4f}")
print(f"AMI: {ami_affinity:.4f}")

resultado_affinity = {
    'Algoritmo': 'AffinityPropagation',
    'N_Clusters': n_clusters_affinity,
    'Silhouette': sil_affinity,
    'Calinski_Harabasz': ch_affinity,
    'ARI': ari_affinity,
    'AMI': ami_affinity
}

Clusters: 758
Silhouette: -0.0275
Calinski-Harabasz: 165.92
ARI: 0.0116
AMI: 0.0601


### Comparativo:

Resultados:

In [52]:
df_resultados = pd.DataFrame([
    resultado_kmeans,
    resultado_dbscan, 
    resultado_agglo,
    resultado_meanshift,
    resultado_affinity
])

df_resultados = df_resultados.sort_values('ARI', ascending=False)

df_resultados

,Algoritmo,N_Clusters,Silhouette,Calinski_Harabasz,ARI,AMI
4,AffinityPropagation,758,-0.027454,165.916547,0.011584,0.060141
1,DBSCAN,1,-1.000000,-1.000000,0.007352,0.086333
3,MeanShift,485,0.473412,161.872499,0.003123,0.039841
2,Agglomerative,15,0.771614,22.739295,0.001435,0.031704
0,KMeans,15,0.757765,16.257534,0.000096,0.001024


Melhor ARI:

In [54]:
melhor_ari = df_resultados.iloc[0]
melhor_ari


Algoritmo            AffinityPropagation
N_Clusters                           758
Silhouette                     -0.027454
Calinski_Harabasz             165.916547
ARI                             0.011584
AMI                             0.060141
Name: 4, dtype: object

Melhor Silhouttte:

In [55]:
melhor_silhouette = df_resultados.loc[df_resultados['Silhouette'].idxmax()]
melhor_silhouette

Algoritmo            Agglomerative
N_Clusters                      15
Silhouette                0.771614
Calinski_Harabasz        22.739295
ARI                       0.001435
AMI                       0.031704
Name: 2, dtype: object

Melhor AMI:

In [56]:
melhor_ami = df_resultados.loc[df_resultados['AMI'].idxmax()]
melhor_ami

Algoritmo              DBSCAN
N_Clusters                  1
Silhouette               -1.0
Calinski_Harabasz        -1.0
ARI                  0.007352
AMI                  0.086333
Name: 1, dtype: object